In [1]:
from datetime import datetime

from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter as draw
from pytket.utils.operators import QubitPauliOperator
from pytket.partition import measurement_reduction, MeasurementBitMap, MeasurementSetup, PauliPartitionStrat
from pytket.backends.backendresult import BackendResult
from pytket.pauli import Pauli, QubitPauliString
from pytket.circuit import Qubit

from scipy.optimize import minimize
from numpy import ndarray
from numpy.random import random_sample
from sympy import Symbol

import qnexus as qnx
from qnexus.references import CircuitRef

# Example VQE workflow using Quantinuum Nexus


VQE example adapted from https://github.com/CQCL/pytket-quantinuum/blob/develop/examples/Quantinuum_variational_experiment_with_batching.ipynb

## Set up the VQE components

In [2]:
# 1. Synthesise Symbolic State-Preparation Circuit (hardware efficient ansatz)

symbols = [Symbol(f"p{i}") for i in range(4)]
symbolic_circuit = Circuit(2)
symbolic_circuit.X(0)
symbolic_circuit.Ry(symbols[0], 0).Ry(symbols[1], 1)
symbolic_circuit.CX(0, 1)
symbolic_circuit.Ry(symbols[2], 0).Ry(symbols[3], 0)

[X q[0]; Ry(p1) q[1]; Ry(p0) q[0]; CX q[0], q[1]; Ry(p2) q[0]; Ry(p3) q[0]; ]

In [3]:
draw(symbolic_circuit)

In [4]:
# 2. Define Hamiltonian 
# coefficients in the Hamiltonian are obtained from PhysRevX.6.031007

coeffs = [-0.4804, 0.3435, -0.4347, 0.5716, 0.0910, 0.0910]
term0 = {
    QubitPauliString(
        {
            Qubit(0): Pauli.I,
            Qubit(1): Pauli.I,
        }
    ): coeffs[0]
}
term1 = {QubitPauliString({Qubit(0): Pauli.Z, Qubit(1): Pauli.I}): coeffs[1]}
term2 = {QubitPauliString({Qubit(0): Pauli.I, Qubit(1): Pauli.Z}): coeffs[2]}
term3 = {QubitPauliString({Qubit(0): Pauli.Z, Qubit(1): Pauli.Z}): coeffs[3]}
term4 = {QubitPauliString({Qubit(0): Pauli.X, Qubit(1): Pauli.X}): coeffs[4]}
term5 = {QubitPauliString({Qubit(0): Pauli.Y, Qubit(1): Pauli.Y}): coeffs[5]}
term_sum = {}
term_sum.update(term0)
term_sum.update(term1)
term_sum.update(term2)
term_sum.update(term3)
term_sum.update(term4)
term_sum.update(term5)
hamiltonian = QubitPauliOperator(term_sum)


In [5]:
# 3 Computing Expectation Values

# Computing Expectation Values for Pauli-Strings
def compute_expectation_paulistring(
    distribution: dict[tuple[int, ...], float], bitmap: MeasurementBitMap
) -> float:
    value = 0
    for bitstring, probability in distribution.items():
        value += probability * (sum(bitstring[i] for i in bitmap.bits) % 2)
    return ((-1) ** bitmap.invert) * (-2 * value + 1)

In [6]:
# 3.2 Computing Expectation Values for sums of Pauli-strings multiplied by coefficients
def compute_expectation_value(
    results: list[BackendResult],
    measurement_setup: MeasurementSetup,
    operator: QubitPauliOperator,
) -> float:
    energy = 0
    for pauli_string, bitmaps in measurement_setup.results.items():
        string_coeff = operator.get(pauli_string, 0.0)
        if string_coeff > 0:
            for bm in bitmaps:
                index = bm.circ_index
                distribution = results[index].get_distribution()
                value = compute_expectation_paulistring(distribution, bm)
                energy += complex(value * string_coeff).real
    return energy

In [7]:
# 4. Building our Objective function

class Objective:
    def __init__(
        self,
        symbolic_circuit: CircuitRef,
        problem_hamiltonian: QubitPauliOperator,
        n_shots_per_circuit: int,
        target: qnx.BackendConfig,
        iteration_number: int = 0,
        n_iterations: int = 10,
    ) -> None:
        r"""Returns the objective function needed for a variational
        procedure.
        """
        terms = [term for term in problem_hamiltonian._dict.keys()]
        self._symbolic_circuit: Circuit = symbolic_circuit.download_circuit()
        self._hamiltonian: QubitPauliOperator = problem_hamiltonian
        self._nshots: int = n_shots_per_circuit
        self._measurement_setup: MeasurementSetup = measurement_reduction(
            terms, strat=PauliPartitionStrat.CommutingSets
        )
        self._iteration_number: int = iteration_number
        self._niters: int = n_iterations
        self._target = target


    def __call__(self, parameter: ndarray) -> float:
        value = self._objective_function(parameter)
        self._iteration_number += 1
        if self._iteration_number >= self._niters:
            self._iteration_number = 0
        return value
    
    def _objective_function(
        self,
        parameters: ndarray,
    ) -> float:
        assert len(parameters) == len(self._symbolic_circuit.free_symbols())
        circuit_list = self._build_circuits(parameters)

        # Label each job with the properties associated with the circuit.
        symbol_dict = {s: p for s, p in zip(self._symbolic_circuit.free_symbols(), parameters)}
        properties = {str(sym): val for sym, val in symbol_dict.items()} | {"iteration": self._iteration_number}

        with qnx.context.using_properties(**properties):
            # Execute circuits with Nexus
            execute_job_ref = qnx.execute(
                name=f"execute_job_VQE_{datetime.now()}_{self._iteration_number}",
                circuits=circuit_list,
                n_shots=[self._nshots]*len(circuit_list),
                target=self._target,
            )

        qnx.job.wait_for(execute_job_ref)
        results = [item.download_result() for item in qnx.job.results(execute_job_ref)]

        expval = compute_expectation_value(
            results, self._measurement_setup, self._hamiltonian
        )
        return expval

    def _build_circuits(self, parameters: ndarray) -> list[CircuitRef]:
        symbol_dict = {s: p for s, p in zip(self._symbolic_circuit.free_symbols(), parameters)}
        properties = {str(sym): val for sym, val in symbol_dict.items()} | {"iteration": self._iteration_number}

        circuit = self._symbolic_circuit.copy()
        circuit.symbol_substitution(symbol_dict)

        with qnx.context.using_properties(**properties):

            # Upload the numerical state-prep circuit to Nexus
            qnx.circuit.upload(
                circuit=circuit,
                name=f"state prep circuit {self._iteration_number}",
            )
            circuit_list = []
            for mc in self._measurement_setup.measurement_circs:
                c = circuit.copy()
                c.append(mc)
                # Upload each measurement circuit to Nexus with correct params
                measurement_circuit_ref = qnx.circuit.upload(
                    circuit=c, 
                    name=f"state prep circuit {self._iteration_number}",
                )
                circuit_list.append(measurement_circuit_ref)

            # Compile circuits with Nexus
            compile_job_ref = qnx.compile(
                name=f"compile_job_VQE_{datetime.now()}_{self._iteration_number}",
                circuits=circuit_list,
                optimisation_level=2,
                target=self._target,
            )
            qnx.job.wait_for(compile_job_ref)
            compile_job_refs = qnx.job.results(compile_job_ref)
            compiled_circuit_refs = [item.get_output() for item in compile_job_refs]
            
        return compiled_circuit_refs

## Set up the Nexus Project and run the VQE

In [8]:
# set up the project
project_ref = qnx.project.create(
    name=f"VQE_example_{str(datetime.now())}",
    description="A VQE done with qnexus",
)

# set this in the context
qnx.context.set_active_project(project_ref)

<Token var=<ContextVar name='qnexus_project' default=None at 0x7f1d372cc360> at 0x7f1d705c0240>

### Introducing Properties

Properties are a way to annotate resources in Nexus with custom attributes.

As we will be computing properties in a loop, the iteration number is a natural fit for the property.

In [9]:
qnx.project.add_property(
    name="iteration", 
    property_type="int", 
    description="The iteration number in my dihydrogen VQE experiment", 
)

In [10]:
# Set up the properties
for sym in symbolic_circuit.free_symbols():
    qnx.project.add_property(
        name=str(sym), 
        property_type="float",
        description=f"Our VQE {str(sym)} parameter", 
    )

In [11]:
# Upload our ansatz circuit

ansatz_ref = qnx.circuit.upload(
    circuit=symbolic_circuit,
    name="ansatz_circuit",
    description="The ansatz state-prep circuit for my dihydrogen VQE",
)

## Construct our objective function

In [12]:
objective = Objective(
    symbolic_circuit = ansatz_ref,
    problem_hamiltonian = hamiltonian,
    n_shots_per_circuit = 500,
    n_iterations= 4,
    target = qnx.QuantinuumConfig(device_name="H1-1LE")
)

In [13]:
initial_parameters = random_sample(len(symbolic_circuit.free_symbols()))

result = minimize(
    objective,
    initial_parameters,
    method="COBYLA",
    options={"disp": True, "maxiter": objective._niters},
    tol=1e-2,
)

print(result.fun)
print(result.x)

-0.45107600000000014

[0.04702527 0.7113198  0.26346873 0.96335837]
   Return from subroutine COBYLA because the MAXFUN limit has been reached.

   NFVALS =    4   F =-4.510760E-01    MAXCV = 0.000000E+00
   X = 4.702527E-02   7.113198E-01   2.634687E-01   9.633584E-01


# Use Nexus to Rescue a VQE workflow

For instance, lets say that some failure happened on the 2nd iteration (e.g. laptop ran out of battery) and we want to resume ASAP.

In the above we ran for 4 iterations, lets pretend that we actually wanted to run for 7 and it failed on the 4th one.

N.B. Assuming the minimizer has no internal state (or has been pickled).

In [14]:
# Get the project, fetching the latest one with the name prefix from above
project_matches = qnx.project.get(name_like="VQE_example_", sort=['-timestamps.created'])

project_ref = project_matches.list()[0]

# set this in the context
qnx.context.set_active_project(project_ref)

project_ref.df()

,name,description,created,modified,contents_modified,id
0,VQE_example_2024-05-27 22:52:54.556172,A VQE done with qnexus,2024-05-27 22:52:55.038677+00:00,2024-05-27 22:52:55.038677+00:00,2024-05-27 22:53:43.775714+00:00,166ac5b0-7848-4be6-bfed-1a34eaf9c61a


In [15]:
# Get the symbolic circuit
symbolic_circuit_ref = qnx.circuit.get_only(name_like="ansatz_circuit")

In [16]:
most_recent_circuits = qnx.circuit.get(name_like="final", project_ref=project_ref)

most_recent_circuits.summarize()

,resource,total_count
0,Circuit,8


In [17]:
most_recent_circuits_refs = most_recent_circuits.list()

most_recent_circuits_refs.df()

,name,description,created,modified,iteration,p0,p1,p2,p3,project,id
0,state prep circuit 0-QuantinuumBackend-final,None,2024-05-27 22:53:01.519283+00:00,2024-05-27 22:53:01.519283+00:00,0,0.71132,0.263469,0.047025,0.963358,VQE_example_2024-05-27 22:52:54.556172,a3baf541-ed69-4946-9ea0-5dde64d69100
1,state prep circuit 0-QuantinuumBackend-final,None,2024-05-27 22:53:01.863898+00:00,2024-05-27 22:53:01.863898+00:00,0,0.71132,0.263469,0.047025,0.963358,VQE_example_2024-05-27 22:52:54.556172,7bfb3f66-5080-4730-8acf-59d3b34f6976
2,state prep circuit 1-QuantinuumBackend-final,None,2024-05-27 22:53:13.238247+00:00,2024-05-27 22:53:13.238247+00:00,1,0.71132,0.263469,1.047025,0.963358,VQE_example_2024-05-27 22:52:54.556172,22662d5e-0982-4e56-8ff7-237dfdaa9e24
3,state prep circuit 1-QuantinuumBackend-final,None,2024-05-27 22:53:13.508730+00:00,2024-05-27 22:53:13.508730+00:00,1,0.71132,0.263469,1.047025,0.963358,VQE_example_2024-05-27 22:52:54.556172,3e062b16-994d-4a1f-a99d-409bcee2c52a
4,state prep circuit 2-QuantinuumBackend-final,None,2024-05-27 22:53:25.043278+00:00,2024-05-27 22:53:25.043278+00:00,2,1.71132,0.263469,0.047025,0.963358,VQE_example_2024-05-27 22:52:54.556172,c364d368-5670-46eb-892f-0791b00608f1
5,state prep circuit 2-QuantinuumBackend-final,None,2024-05-27 22:53:25.466271+00:00,2024-05-27 22:53:25.466271+00:00,2,1.71132,0.263469,0.047025,0.963358,VQE_example_2024-05-27 22:52:54.556172,67702eb8-3df1-4d1c-a64f-04d7d9548ec2
6,state prep circuit 3-QuantinuumBackend-final,None,2024-05-27 22:53:37.143999+00:00,2024-05-27 22:53:37.143999+00:00,3,0.71132,1.263469,0.047025,0.963358,VQE_example_2024-05-27 22:52:54.556172,5820dab2-80e5-444c-a614-4b067f3fd968
7,state prep circuit 3-QuantinuumBackend-final,None,2024-05-27 22:53:37.479547+00:00,2024-05-27 22:53:37.479547+00:00,3,0.71132,1.263469,0.047025,0.963358,VQE_example_2024-05-27 22:52:54.556172,26682e5b-3d7e-4a65-9a19-dab0863668df


In [18]:
# Get the latest circuit to get the new 'initial_parameters'
latest_circuit: CircuitRef = most_recent_circuits_refs[-1]

latest_circuit_properties = latest_circuit.annotations.properties

latest_circuit.df()

,name,description,created,modified,iteration,p0,p1,p2,p3,project,id
0,state prep circuit 3-QuantinuumBackend-final,None,2024-05-27 22:53:37.479547+00:00,2024-05-27 22:53:37.479547+00:00,3,0.71132,1.263469,0.047025,0.963358,VQE_example_2024-05-27 22:52:54.556172,26682e5b-3d7e-4a65-9a19-dab0863668df


In [19]:
# Get what iteration we were on (from the latest circuit)

last_iteration_count = latest_circuit_properties.pop("iteration")

print(last_iteration_count)

# Retreive the params and check them
new_starting_params = list(latest_circuit_properties.values())
print(new_starting_params)


3
[0.7113198041915894, 1.2634687423706055, 0.047025274485349655, 0.9633583426475525]


In [20]:
# Build the Objective and run 'minimize' to continue the experiment
objective = Objective(
    symbolic_circuit_ref,
    hamiltonian,
    n_shots_per_circuit = 500,
    iteration_number=last_iteration_count, # resume from 3rd iteration of 7
    n_iterations = 7,
    target = qnx.QuantinuumConfig(device_name="H1-1LE")
)

result = minimize(
    objective,
    new_starting_params,
    method="COBYLA",
    options={"disp": True, "maxiter": objective._niters},
    tol=1e-2,
)

print(result.fun)
print(result.x)

-0.5347836
   Return from subroutine COBYLA because the MAXFUN limit has been reached.

   NFVALS =    7   F =-5.347836E-01    MAXCV = 0.000000E+00
   X = 1.711320E+00   1.263469E+00   4.702527E-02   9.633583E-01

[1.7113198  1.26346874 0.04702527 0.96335834]


### Querying by property

In [21]:
qnx.circuit.get(properties={"iteration": 20}).df()

,name,description,created,modified,project,id,iteration,p0,p1,p2,p3
0,ansatz_circuit,The ansatz state-prep circuit for my dihydroge...,2024-05-27 22:52:55.406473+00:00,2024-05-27 22:52:55.406473+00:00,VQE_example_2024-05-27 22:52:54.556172,b1538afb-76c7-4f39-82a2-03e9a579f73e,NaN,NaN,NaN,NaN,NaN
1,state prep circuit 0,None,2024-05-27 22:52:55.517401+00:00,2024-05-27 22:52:55.517401+00:00,VQE_example_2024-05-27 22:52:54.556172,7a1a6825-3f51-43f6-a566-94e29dafa74f,0.0,0.711320,0.263469,0.047025,0.963358
2,state prep circuit 0,None,2024-05-27 22:52:55.599857+00:00,2024-05-27 22:52:55.599857+00:00,VQE_example_2024-05-27 22:52:54.556172,f21a2732-b45d-4c7b-b1b0-a826c27747a1,0.0,0.711320,0.263469,0.047025,0.963358
3,state prep circuit 0,None,2024-05-27 22:52:55.667992+00:00,2024-05-27 22:52:55.667992+00:00,VQE_example_2024-05-27 22:52:54.556172,32e24fd0-6564-43ab-b3b8-326c07d647df,0.0,0.711320,0.263469,0.047025,0.963358
4,state prep circuit 0-QuantinuumBackend-2,None,2024-05-27 22:53:01.384768+00:00,2024-05-27 22:53:01.384768+00:00,VQE_example_2024-05-27 22:52:54.556172,54dd9236-ba7b-41c4-937c-1b7694f65c20,0.0,0.711320,0.263469,0.047025,0.963358
...,...,...,...,...,...,...,...,...,...,...,...
139,state prep circuit 2-QuantinuumBackend-2,None,2024-05-27 22:55:01.541265+00:00,2024-05-27 22:55:01.541265+00:00,VQE_example_2024-05-27 22:52:54.556172,b5491743-1411-4105-82ed-ce7268defda7,2.0,1.103887,-0.188601,2.006794,0.677511
140,state prep circuit 2-QuantinuumBackend-3,None,2024-05-27 22:55:01.572942+00:00,2024-05-27 22:55:01.572942+00:00,VQE_example_2024-05-27 22:52:54.556172,ccdc735b-42de-428e-9644-d4b19e7cf0e6,2.0,1.103887,-0.188601,2.006794,0.677511
141,state prep circuit 2-QuantinuumBackend-4,None,2024-05-27 22:55:01.606715+00:00,2024-05-27 22:55:01.606715+00:00,VQE_example_2024-05-27 22:52:54.556172,7385f528-b52b-4cf8-a4e1-05d07857fe67,2.0,1.103887,-0.188601,2.006794,0.677511
142,state prep circuit 2-QuantinuumBackend-5,None,2024-05-27 22:55:01.642987+00:00,2024-05-27 22:55:01.642987+00:00,VQE_example_2024-05-27 22:52:54.556172,c2f3e7c3-fe37-40c9-a54e-2248bdc3d66c,2.0,1.103887,-0.188601,2.006794,0.677511
